## G2P로 생성된 노이즈 데이터를 제거하는 파일

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("../data/train.csv")
train_df.head()

,ID,text,target,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
3,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54
4,ynat-v1_train_00004,회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05


In [3]:
len(train_df)

45678

In [5]:
# 중복 행 제거
train_df.drop_duplicates(subset=["text", "target"], inplace=True, ignore_index=True)

In [6]:
len(train_df)

45678

In [7]:
# pip install git+https://github.com/jungin500/py-hanspell
# spell_checker.py에서 class:'purple'을 class:'violet'으로 수정
from hanspell import spell_checker
from tqdm.notebook import tqdm

In [10]:
checked = []
for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
    checked.append(spell_checker.check(row["text"]).as_dict())

  0%|          | 0/45678 [00:00<?, ?it/s]

In [11]:
train_df["spell_check"] = checked

In [12]:
def calc_error_ratio(error_dict):
    word_list = list(error_dict["words"].values())
    total_len = len(word_list)
    not_error_word = word_list.count(0) + word_list.count(2) # 0번은 맞춤법 문제 없음, 2번은 띄어쓰기 오류. 정상 문장에서도 띄어쓰기 오류는 많아서 제외함.

    return (total_len - not_error_word) / total_len

In [13]:
train_df["error_ratio"] = train_df["spell_check"].apply(calc_error_ratio)

In [16]:
# 보통 error ratio가 0.2를 넘어가면 이상한 문장임
train_df[(train_df["error_ratio"] > 0.2) & (train_df["error_ratio"] < 0.25)].sample(10)

,ID,text,target,url,date,spell_check,error_ratio
30772,ynat-v1_train_30772,에이스 이재영이 오세트에 외친 말 라 줘 나 줘,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.03.25 22:33,"{'result': True, 'original': '에이스 이재영이 오세트에 외친...",0.222222
12453,ynat-v1_train_12453,삼성重·대우조선 파산보호 신청한 시드릴과 계약해지종합,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.26. 오후 6:16,"{'result': True, 'original': '삼성重·대우조선 파산보호 신청...",0.222222
26670,ynat-v1_train_26670,첼로 거장 마이스키 절믄 클래시 강상블 디토와 한 무대,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 11:29,"{'result': True, 'original': '첼로 거장 마이스키 절믄 클래...",0.222222
20970,ynat-v1_train_20970,스스로 자부시믈 로피자…한화 워크쇼브로 시즌 마무리,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.11.28 17:57,"{'result': True, 'original': '스스로 자부시믈 로피자…한화 ...",0.222222
44843,ynat-v1_train_44843,특징주 엘앤케이바이오 코스닥 이전 상장 첫날 급락종합,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.30. 오후 4:01,"{'result': True, 'original': '특징주 엘앤케이바이오 코스닥 ...",0.222222
1687,ynat-v1_train_01687,그래피 견말정사 나라두며 뉴용한 사가지,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.30. 오후 12:00,"{'result': True, 'original': '그래피 견말정사 나라두며 뉴용...",0.222222
997,ynat-v1_train_00997,엘드림 학교 꿈을 찾아 떠나는 축구 캠프 개최,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.12.29 12:14,"{'result': True, 'original': '엘드림 학교 꿈을 찾아 떠나는...",0.222222
5037,ynat-v1_train_05037,北매체 함경북도 수해복구서 전화위복 승리 거둬종합,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.12.07. 오후 6:10,"{'result': True, 'original': '北매체 함경북도 수해복구서 전...",0.222222
19619,ynat-v1_train_19619,靑 판문점선언 이행추진위 오늘 3차 회의…후속조치 점검,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.19. 오전 9:37,"{'result': True, 'original': '靑 판문점선언 이행추진위 오늘...",0.222222
43802,ynat-v1_train_43802,통일부 차관 北도발 대처위해 궁미녕냥 결집 쭝요,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.24. 오후 3:09,"{'result': True, 'original': '통일부 차관 北도발 대처위해 ...",0.222222


In [17]:
G2P_cleaned_df = train_df[train_df["error_ratio"] < 0.2].drop(["spell_check", "error_ratio"], axis=1)
len(G2P_cleaned_df)

40738

In [18]:
G2P_cleaned_df.to_csv("../data/G2P_cleaned_train.csv", index=False)

총 데이터 갯수 45678 -> 47253개 (4940개 데이터 삭제)